In [153]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

from datetime import datetime, timedelta

from statistics import mean

from sklearn.decomposition import PCA

import requests
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from bs4 import BeautifulSoup

**Aim: For each location in the dataset, find the 5 most similar locations/tourist destinations based on the selected variables.**

In [154]:
random.seed(3888)

# Preparing the data for nearest neighbours analysis

## Data import

In [155]:
df = pd.read_csv("data/data.txt")
df = df.drop(columns='Unnamed: 0')

df = df.rename(columns = {'tourist_service_index': 'tourist_service_infrastructure'})

# An Australian wouldn't travel to Australia!

df = df[df["iso_code"] != "AUS"]

In [156]:
df_without_covid = df.drop(columns=['new_cases_per_million', 
                                    'new_cases_smoothed_per_million', 
                                    'stringency_index', 
                                    'positive_rate', 
                                    'human_development_index', 
                                    'international_travel_controls',
                                    'cost_living_index',
                                    'date'])

In [157]:
non_covid_colnames = ['iso_code', 'location', 'continent', 'tourist_service_infrastructure', 'art_gallery']
covid_colnames = ['iso_code', 'new_cases_smoothed_per_million', 'date']

In [158]:
df_without_covid = df_without_covid.drop_duplicates()
df_without_covid = df_without_covid.reset_index()
df_without_covid = df_without_covid.drop(columns=['index'])

df_without_covid

,iso_code,location,continent,tourist_service_infrastructure
0,ALB,Albania,Europe,4.01
1,DZA,Algeria,Africa,1.84
2,ARG,Argentina,South America,4.53
3,ARM,Armenia,Asia,4.33
4,AUT,Austria,Europe,6.66
...,...,...,...,...
110,VEN,Venezuela,South America,2.96
111,VNM,Vietnam,Asia,2.85
112,YEM,Yemen,Asia,1.87
113,ZMB,Zambia,Africa,2.52


In [159]:
full_tourism = pd.read_csv("data/full_tourism.csv")
full_tourism = full_tourism[full_tourism["Country ISO3"] != "AUS"]

In [160]:
indicators = {
                'WEF Infrastructure subindex, 1-7 (best)': 'infrastructure', 
                'WEF Natural and cultural resources subindex, 1-7 (best)': 'natural_cultural_resources',
                'WEF Safety and security pillar, 1-7 (best)': 'safety_security',
                'WEF Health and hygiene, 1-7 (best)': 'health_hygiene',
                'WEF Price competitiveness in the Travel and Tourism Industry pillar, 1-7 (best)': 'price_competitiveness',
                'WEF Air transport infrastructure, 1-7 (best)': 'air_transport',
                'WEF Ground and port infrastructure, 1-7 (best)': 'ground_port'
}

full_tourism_req_indicators = full_tourism[full_tourism["Indicator"].isin(indicators)]
full_tourism_req_indicators = full_tourism_req_indicators[['Country ISO3', 'Indicator', 'Subindicator Type', '2019']]
full_tourism_req_indicators = full_tourism_req_indicators[full_tourism["Subindicator Type"] == "Value"]
full_tourism_req_indicators = full_tourism_req_indicators.drop(columns = ['Subindicator Type'])
full_tourism_req_indicators = full_tourism_req_indicators.rename(columns = {'Country ISO3': 'iso_code'})
full_tourism_req_indicators = full_tourism_req_indicators.set_index('iso_code')

inds = pd.DataFrame()

for ind in indicators.keys():
    inds[indicators[ind]] = full_tourism_req_indicators[full_tourism_req_indicators["Indicator"] == ind].drop(columns = ["Indicator"]).rename(columns = {'2019': indicators[ind]})[indicators[ind]]
    
non_covid = pd.merge(inds, df_without_covid, on='iso_code')

/tmp/ipykernel_6048/613716665.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  full_tourism_req_indicators = full_tourism_req_indicators[full_tourism["Subindicator Type"] == "Value"]


In [161]:
covid = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

covid = covid[covid["iso_code"] != "AUS"]

In [162]:
covid_needed_cols = covid[covid_colnames]

In [163]:
covid_needed_cols['date'] = pd.to_datetime(covid_needed_cols['date'], format='%Y-%m-%d')

/tmp/ipykernel_6048/1566742333.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_needed_cols['date'] = pd.to_datetime(covid_needed_cols['date'], format='%Y-%m-%d')


In [164]:
covid_needed_cols = covid_needed_cols[covid_needed_cols['date'] >= datetime.now() - timedelta(days = 30)]

In [165]:
poi = pd.read_json("data/poi_types.json")

In [166]:
poi = poi.fillna(0)
poi = poi.replace(0, np.nan)
poi = poi.dropna(how='all', axis=0)
poi = poi.replace(np.nan, 0)

In [167]:
poi = poi.transpose()

In [168]:
poi = poi[poi.index != "Australia"]

In [169]:
iso_location = df[["iso_code", "location"]].drop_duplicates()

iso_location.to_csv("iso_location.csv")

In [170]:
def iso_code_to_loc(iso_code):
    return iso_location[iso_location["iso_code"] == iso_code]["location"].iloc[0]

In [171]:
def loc_to_iso_code(loc):
    return iso_location[iso_location["location"] == loc]["iso_code"].iloc[0]

In [172]:
poi = poi.set_index(loc_to_iso_code(loc) for loc in poi.index)

In [173]:
poi.columns

Index(['art_gallery', 'food', 'museum', 'park', 'restaurant', 'church',
       'mosque', 'place_of_worship', 'zoo', 'travel_agency', 'amusement_park',
       'aquarium', 'cafe', 'store', 'cemetery', 'hindu_temple',
       'natural_feature', 'library', 'campground', 'lodging', 'casino',
       'local_government_office', 'liquor_store', 'bar', 'shopping_mall',
       'spa', 'transit_station', 'grocery_or_supermarket', 'synagogue',
       'movie_theater', 'general_contractor', 'parking', 'book_store',
       'night_club', 'city_hall', 'clothing_store', 'department_store',
       'health', 'hospital'],
      dtype='object')

In [174]:
# removing less relevant columns

poi = poi.drop(columns=['travel_agency',
                        'store',
                        'cemetery',
                        'library',
                        'campground',
                        'lodging',
                        'local_government_office',
                        'liquor_store',
                        'transit_station',
                        'grocery_or_supermarket',
                        'movie_theater',
                        'general_contractor',
                        'parking',
                        'book_store',
                        'city_hall',
                        'health',
                        'hospital'
                       ])

In [175]:
poi['iso_code'] = poi.index

In [176]:
covid_merged = pd.merge(covid_needed_cols, non_covid, on="iso_code")
covid_merged = pd.merge(covid_merged, poi, on="iso_code")

In [177]:
# https://practicaldatascience.co.uk/data-science/how-to-read-an-rss-feed-in-python

def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)


def get_feed(url):
    """Return a Pandas dataframe containing the RSS feed contents.

    Args: 
        url (string): URL of the RSS feed to read.

    Returns:
        df (dataframe): Pandas dataframe containing the RSS feed contents.
    """
    
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'guid', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            guid = item.find('guid', first=True).text
            description = item.find('description', first=True).text

            row = {'title': [title], 'pubDate': [pubDate], 'guid': [guid], 'description': [description]}
            df = pd.concat([df, pd.DataFrame.from_dict(row)])

    return df

In [178]:
travel_advice = get_feed("https://www.smartraveller.gov.au/countries/documents/index.rss")

In [179]:
travel_advice = travel_advice[travel_advice["title"] != "No travel advice"]

In [180]:
travel_advice = travel_advice.drop(columns=['guid'])

In [181]:
travel_advice

,title,pubDate,description
0,Mali,07 May 2022 22:00:00 AEST,"On 6 May 2022, the US government issued an ale..."
0,Sri Lanka,07 May 2022 22:00:00 AEST,A Public Emergency has been declared in Sri La...
0,Timor-Leste,06 May 2022 22:00:00 AEST,The Presidential inauguration will be held on ...
0,Papua New Guinea,06 May 2022 22:00:00 AEST,"If you&#039;re not a PNG citizen, you&#039;ll ..."
0,Hong Kong,06 May 2022 22:00:00 AEST,Fully vaccinated travellers can enter Hong Kon...
...,...,...,...
0,Costa Rica,28 Oct 2021 23:00:00 AEDT,If you&#039;re not fully vaccinated against CO...
0,Marshall Islands,28 Oct 2021 23:00:00 AEDT,"Due to COVID-19, Marshall Islands has closed i..."
0,Niger,28 Oct 2021 23:00:00 AEDT,The US Government has issued a security alert ...
0,North Korea (Democratic People's Republic of K...,28 Oct 2021 23:00:00 AEDT,North Korea’s borders remain closed due to COV...


In [182]:
replacements = {
    "United States of America": "United States",
    "Israel and the Palestinian Territories": "Israel",
    "South Korea (Republic of Korea)": "South Korea"
}

In [183]:
for replacement in replacements:
    travel_advice.replace(replacement, replacements[replacement], inplace = True)

In [184]:
set(covid_merged["location"]).difference(set(travel_advice["title"]))

{'Barbados', 'Luxembourg', 'Suriname'}

In [185]:
travel_advice.rename(columns={"title": "location", "description": "advice"}, inplace = True)

In [186]:
travel_advice["advice"] = [BeautifulSoup(s, "lxml").text for s in travel_advice["advice"]]

In [187]:
covid_merged = pd.merge(covid_merged, travel_advice, on="location")

In [188]:
covid_merged

,iso_code,new_cases_smoothed_per_million,date,infrastructure,natural_cultural_resources,safety_security,health_hygiene,price_competitiveness,air_transport,ground_port,...,casino,bar,shopping_mall,spa,synagogue,night_club,clothing_store,department_store,pubDate,advice
0,ALB,18.299,2022-04-09,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
1,ALB,17.354,2022-04-10,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
2,ALB,15.216,2022-04-11,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
3,ALB,13.525,2022-04-12,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
4,ALB,13.177,2022-04-13,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,ZWE,3.190,2022-05-02,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...
3139,ZWE,3.057,2022-05-03,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...
3140,ZWE,3.038,2022-05-04,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...
3141,ZWE,2.518,2022-05-05,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...


In [189]:
covid_colnames.append("advice")

covid_merged = covid_merged[list(set(non_covid_colnames).union(set(covid_colnames)))]

covid_colnames.remove("advice")

covid_merged

,date,advice,location,continent,iso_code,art_gallery,new_cases_smoothed_per_million,tourist_service_infrastructure
0,2022-04-09,Face coverings must be worn at all times in in...,Albania,Europe,ALB,3.0,18.299,4.01
1,2022-04-10,Face coverings must be worn at all times in in...,Albania,Europe,ALB,3.0,17.354,4.01
2,2022-04-11,Face coverings must be worn at all times in in...,Albania,Europe,ALB,3.0,15.216,4.01
3,2022-04-12,Face coverings must be worn at all times in in...,Albania,Europe,ALB,3.0,13.525,4.01
4,2022-04-13,Face coverings must be worn at all times in in...,Albania,Europe,ALB,3.0,13.177,4.01
...,...,...,...,...,...,...,...,...
3138,2022-05-02,We now advise you exercise a high degree of ca...,Zimbabwe,Africa,ZWE,0.0,3.190,2.95
3139,2022-05-03,We now advise you exercise a high degree of ca...,Zimbabwe,Africa,ZWE,0.0,3.057,2.95
3140,2022-05-04,We now advise you exercise a high degree of ca...,Zimbabwe,Africa,ZWE,0.0,3.038,2.95
3141,2022-05-05,We now advise you exercise a high degree of ca...,Zimbabwe,Africa,ZWE,0.0,2.518,2.95


In [190]:
descriptions = pd.read_csv("country_descriptions_cleaned_2.csv")

In [191]:
covid_merged = pd.merge(descriptions, covid_merged, on="iso_code")

In [192]:
covid_merged

,iso_code,description,date,advice,location,continent,art_gallery,new_cases_smoothed_per_million,tourist_service_infrastructure
0,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-09,Face coverings must be worn at all times in in...,Albania,Europe,3.0,18.299,4.01
1,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-10,Face coverings must be worn at all times in in...,Albania,Europe,3.0,17.354,4.01
2,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-11,Face coverings must be worn at all times in in...,Albania,Europe,3.0,15.216,4.01
3,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-12,Face coverings must be worn at all times in in...,Albania,Europe,3.0,13.525,4.01
4,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-13,Face coverings must be worn at all times in in...,Albania,Europe,3.0,13.177,4.01
...,...,...,...,...,...,...,...,...,...
3082,GBR,The United Kingdom of Great Britain and Northe...,2022-05-02,All COVID-19 travel restrictions in the UK hav...,United Kingdom,Europe,6.0,127.586,6.10
3083,GBR,The United Kingdom of Great Britain and Northe...,2022-05-03,All COVID-19 travel restrictions in the UK hav...,United Kingdom,Europe,6.0,166.836,6.10
3084,GBR,The United Kingdom of Great Britain and Northe...,2022-05-04,All COVID-19 travel restrictions in the UK hav...,United Kingdom,Europe,6.0,165.496,6.10
3085,GBR,The United Kingdom of Great Britain and Northe...,2022-05-05,All COVID-19 travel restrictions in the UK hav...,United Kingdom,Europe,6.0,162.132,6.10


## Computing medians of quantitative columns

In [193]:
covid_merged_no_quant = list(set(covid_merged.columns).difference(set(covid_merged.select_dtypes(include=[np.number]).columns)))
covid_merged_no_quant.remove("date")

In [194]:
medians = covid_merged.groupby(["iso_code"]).median()

In [195]:
medians = medians.fillna(covid_merged.median())

/tmp/ipykernel_6048/3187006807.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  medians = medians.fillna(covid_merged.median())
/tmp/ipykernel_6048/3187006807.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  medians = medians.fillna(covid_merged.median())


In [196]:
medians

,art_gallery,new_cases_smoothed_per_million,tourist_service_infrastructure
iso_code,,,
ALB,3.0,15.2655,4.01
ARE,0.0,23.8930,5.63
ARG,0.0,32.9380,4.53
ARM,0.0,3.8265,4.33
AUT,0.0,860.4850,6.66
...,...,...,...
VNM,0.0,208.5280,2.85
YEM,0.0,0.0050,1.87
ZAF,0.0,42.3015,4.30


In [197]:
all_data_w_medians = pd.merge(medians, covid_merged[covid_merged_no_quant], on="iso_code").drop_duplicates()

In [198]:
all_data_w_medians = all_data_w_medians.set_index(all_data_w_medians["iso_code"]).drop(columns=["iso_code"])

In [199]:
all_data_w_medians

,art_gallery,new_cases_smoothed_per_million,tourist_service_infrastructure,advice,location,continent,description
iso_code,,,,,,,
ALB,3.0,15.2655,4.01,Face coverings must be worn at all times in in...,Albania,Europe,Albania (Albanian: Shqipëria) is a country in ...
ARE,0.0,23.8930,5.63,You no longer need a COVID-19 test to travel t...,United Arab Emirates,Asia,The United Arab Emirates (Arabic: دولة الإمارا...
ARG,0.0,32.9380,4.53,You no longer need to have a COVID-19 test or ...,Argentina,South America,Argentina is a large country in the southern p...
ARM,0.0,3.8265,4.33,You can enter Armenia with a negative COVID-19...,Armenia,Asia,Armenia (Armenian: Հայաստան Hayastan) is a lan...
AUT,0.0,860.4850,6.66,Most COVID-19 measures across Austria have bee...,Austria,Europe,"Austria (German: Österreich, literally ""the Ea..."
...,...,...,...,...,...,...,...
VNM,0.0,208.5280,2.85,We now advise you exercise a high degree of ca...,Vietnam,Asia,Vietnam (Vietnamese: Việt Nam) is a country in...
YEM,0.0,0.0050,1.87,We've reviewed our travel advice for Yemen. We...,Yemen,Asia,Yemen (Arabic: ٱلْيَمَن) is a country in the M...
ZAF,0.0,42.3015,4.30,Heavy rains and flooding in the city of Durban...,South Africa,Africa,"SA redirects here. For the state, see South Au..."


## Min-max scaling

In [200]:
iso_code = medians.index

In [201]:
scaler = MinMaxScaler()

In [202]:
medians_scaled = scaler.fit_transform(medians)

In [203]:
cols = list(set(non_covid_colnames).union(set(covid_colnames)))
to_remove = ['iso_code', 'continent', 'location', 'date']

for col in to_remove:
    cols.remove(col)
    
medians_scaled = pd.DataFrame(medians_scaled, 
                              columns = cols, 
                              index = iso_code)
medians_scaled

,art_gallery,new_cases_smoothed_per_million,tourist_service_infrastructure
iso_code,,,
ALB,0.5,0.009083,0.458753
ARE,0.0,0.014217,0.784708
ARG,0.0,0.019598,0.563380
ARM,0.0,0.002277,0.523139
AUT,0.0,0.511997,0.991952
...,...,...,...
VNM,0.0,0.124076,0.225352
YEM,0.0,0.000003,0.028169
ZAF,0.0,0.025170,0.517103


In [204]:
# perform PCA if > 2 attributes selected

if len(medians_scaled.columns) > 2:
    pca = PCA(n_components=2)
    pc = pca.fit_transform(medians_scaled)
    medians_scaled = pd.DataFrame(data = pc, columns = ['PC1', 'PC2'], index = medians_scaled.index)

In [205]:
medians_scaled

,PC1,PC2
iso_code,,
ALB,-0.071229,0.433603
ARE,0.155416,-0.072378
ARG,-0.023346,-0.066951
ARM,-0.066162,-0.063857
AUT,0.606168,-0.135371
...,...,...
VNM,-0.242118,-0.069749
YEM,-0.474003,-0.050072
ZAF,-0.058218,-0.066329


# Finding 5 nearest neighbours for each location

In [206]:
num_neighbours = 5

In [207]:
dist_metrics = ['euclidean', 'manhattan', 'chebyshev', 'cosine', 'cityblock', 'braycurtis', 'canberra',
               'correlation', 'minkowski']

In [208]:
location_neighbours = {}

for metric in dist_metrics:
    nbrs = NearestNeighbors(metric = metric, 
                            n_neighbors = num_neighbours + 1, 
                            algorithm='auto').fit(medians_scaled)
    
    nbr_indices = list(list(x) for x in nbrs.kneighbors(medians_scaled)[1])
    
    iso_location = df[["iso_code", "location"]].drop_duplicates()

    for i in range(len(nbr_indices)):
        current_iso_code = list(medians_scaled.index)[i]
        current_location = iso_code_to_loc(current_iso_code)

        neighbours = []
        for j in range(1, num_neighbours + 1):
            iso_code = medians_scaled.index[nbr_indices[i][j]]
            neighbours.append(iso_code_to_loc(iso_code))

        if not current_location in location_neighbours:
            location_neighbours[current_location] = {metric: neighbours}
        else:
            location_neighbours[current_location][metric] = neighbours

In [209]:
location_neighbours_df = pd.DataFrame(location_neighbours).transpose()

In [210]:
location_neighbours_df

,euclidean,manhattan,chebyshev,cosine,cityblock,braycurtis,canberra,correlation,minkowski
Albania,"[Serbia, Morocco, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Serbia, Morocco, Colombia, Russia, Kazakhstan]","[Serbia, Morocco, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Morocco, Kazakhstan, Lebanon, Iran, Kenya]","[Serbia, Morocco, Colombia, Russia, Kazakhstan]"
United Arab Emirates,"[Montenegro, Costa Rica, Mauritius, Denmark, P...","[Montenegro, Costa Rica, Mauritius, Peru, Norway]","[Montenegro, Costa Rica, Mauritius, Denmark, P...","[Montenegro, Finland, Estonia, Belgium, Israel]","[Montenegro, Costa Rica, Mauritius, Peru, Norway]","[Montenegro, Costa Rica, Mauritius, Peru, Denm...","[Montenegro, Costa Rica, Peru, Mauritius, Bulg...","[France, Greece, Turkey, Japan, Finland]","[Montenegro, Costa Rica, Mauritius, Denmark, P..."
Argentina,"[Brazil, Poland, Chile, Romania, Myanmar]","[Brazil, Poland, Romania, Chile, Myanmar]","[Brazil, Poland, Chile, Romania, Myanmar]","[Brazil, Chile, Poland, Romania, Myanmar]","[Brazil, Poland, Romania, Chile, Myanmar]","[Brazil, Poland, Chile, Romania, Myanmar]","[Brazil, Poland, Chile, Romania, Myanmar]","[South Korea, Italy, Japan, Israel, Jamaica]","[Brazil, Poland, Chile, Romania, Myanmar]"
Armenia,"[Ukraine, South Africa, Myanmar, Tunisia, Chile]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Chile]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Mongolia, Iran, North Macedonia, India, Kenya]","[Ukraine, South Africa, Myanmar, Tunisia, Chile]"
Austria,"[Portugal, Germany, Cyprus, Italy, New Zealand]","[Portugal, Germany, Cyprus, Italy, New Zealand]","[Portugal, Germany, Cyprus, Italy, South Korea]","[Portugal, Spain, United States, Germany, New ...","[Portugal, Germany, Cyprus, Italy, New Zealand]","[Portugal, Germany, Cyprus, Italy, New Zealand]","[Portugal, Italy, Germany, Cyprus, New Zealand]","[Malta, Latvia, Finland, Mexico, Cyprus]","[Portugal, Germany, Cyprus, Italy, New Zealand]"
...,...,...,...,...,...,...,...,...,...
Vietnam,"[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Paraguay, Sri Lanka, Egypt, Cambodia, Bolivia]","[Honduras, Nicaragua, China, Philippines, Boli...","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Pakistan, United Kingdom, Nepal, Ghana, Oman]","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]"
Yemen,"[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Oman, Algeria, Mongolia, Philippines, Ethiopia]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]"
South Africa,"[Myanmar, Armenia, Ukraine, Chile, Poland]","[Myanmar, Armenia, Ukraine, Poland, Chile]","[Myanmar, Armenia, Ukraine, Chile, Poland]","[Myanmar, Armenia, Ukraine, Tunisia, Oman]","[Myanmar, Armenia, Ukraine, Poland, Chile]","[Myanmar, Armenia, Ukraine, Poland, Tunisia]","[Myanmar, Armenia, Ukraine, Tunisia, Oman]","[Qatar, Taiwan, Estonia, France, Mexico]","[Myanmar, Armenia, Ukraine, Chile, Poland]"
Zambia,"[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, Nepal]","[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, Indi

In [211]:
prop_sim = []

for location in location_neighbours.keys():
    ind_pairs = [[0, 1], [0, 2], [1, 2]]
    
    for pair in ind_pairs:
        loc_intersect = set(location_neighbours[location][dist_metrics[0]]).intersection(location_neighbours[location][dist_metrics[1]])
        prop_sim.append(len(loc_intersect)/num_neighbours)

In [212]:
mean(prop_sim)

0.9581818181818182

In [213]:
def find_top_neighbours(country):
    d = {}
    lists = location_neighbours_df.loc[country].tolist()
    for ls in lists:
        for c in ls:
            if c in d:
                d[c] += 1
            else:
                d[c] = 1
    top = []

    for k,v in sorted(d.items(), key=lambda p:p[1], reverse=True)[:num_neighbours]:
        top.append(k)
    return top

In [214]:
locations = location_neighbours_df.index.tolist()
top_neighbours = {}
for location in locations:
    top_neighbours[location] = find_top_neighbours(location)
top_neighbours_df = pd.DataFrame(top_neighbours).transpose()
top_neighbours_df

,0,1,2,3,4
Albania,Morocco,Kazakhstan,Serbia,Colombia,Russia
United Arab Emirates,Montenegro,Costa Rica,Mauritius,Peru,Denmark
Argentina,Brazil,Poland,Chile,Romania,Myanmar
Armenia,Ukraine,South Africa,Myanmar,Tunisia,Oman
Austria,Portugal,Germany,Cyprus,Italy,New Zealand
...,...,...,...,...,...
Vietnam,Bolivia,Sri Lanka,Paraguay,Cambodia,Egypt
Yemen,Algeria,Ethiopia,Bangladesh,Cameroon,Nepal
South Africa,Myanmar,Armenia,Ukraine,Poland,Chile
Zambia,Tanzania,Pakistan,Ghana,Uganda,India


In [216]:
top_neighbours_df.loc["United Kingdom"]

0    Netherlands
1        Croatia
2        Ireland
3       Slovenia
4       Slovakia
Name: United Kingdom, dtype: object